In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [11]:
dat = pd.read_csv(('bearing_prediction_data.csv'))
dat.head()

,Unnamed: 0,x_mean,y_mean,x_std,y_std,x_rms,y_rms,x_sf,y_sf,x_peak_count,y_peak_count,bearing_condition
0,0,-0.090969,-0.094235,0.060086,0.066382,0.109020,0.115267,0.748956,0.796017,13,43,0
1,1,-0.091260,-0.087946,0.059421,0.064746,0.108900,0.109208,0.746547,0.790427,10,29,0
2,2,-0.092597,-0.093106,0.059969,0.065255,0.110319,0.113696,0.741149,0.791236,11,21,0
3,3,-0.093485,-0.093244,0.059917,0.066303,0.111037,0.114413,0.744487,0.787161,5,22,0
4,4,-0.104769,-0.093042,0.058607,0.065404,0.120046,0.113729,0.746573,0.792580,2,24,0


In [23]:
model = smf.ols('bearing_condition ~ x_mean + y_mean + x_std + y_std + x_rms + y_rms + x_sf + y_sf + x_peak_count + y_peak_count', data = dat).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      bearing_condition   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.888
Method:                 Least Squares   F-statistic:                     1362.
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        12:15:03   Log-Likelihood:                -906.72
No. Observations:                1724   AIC:                             1835.
Df Residuals:                    1713   BIC:                             1895.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       11.8095      1.522      7.760   